In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
%matplotlib inline
import seaborn as sns
import itertools

# load data

In [2]:
train= pd.read_csv('train.csv')
validation= pd.read_csv('validation.csv')

# basic statistical information

In [17]:
def basic_stats(df):
    win = pd.DataFrame()
    temp = pd.DataFrame()
    stats = pd.DataFrame()
    stats['Advertiser'] = np.sort(df.advertiser.unique())
    stats['Impressions'] = df.groupby('advertiser').size().values    
    click = df.groupby('advertiser').click.value_counts() 
    stats['Clicks'] = click.iloc[click.index.get_level_values('click') == 1].values
    click_w = df.loc[( df['bidprice']>= df['payprice']) & (df['click']==1 )].groupby('advertiser')
    click2 = df.loc[( df['bidprice']>= df['payprice'])].groupby('advertiser')
    cost_click = click2.sum()[['click','payprice']]
    temp['Cost'] = (cost_click['payprice']).tolist()
    ## Cost
    stats['Cost'] = ((cost_click['payprice']/1000).round(1)).tolist()
    stats['CTR'] = (((stats.Clicks / stats.Impressions) * 100).round(3)).astype(str) + '%'
    ##eCPC
    stats['eCPC'] = ((temp.Cost / (stats.Clicks*1000 )).round(2)).astype(str)
    ##Avg CPM
    win['Cost'] = df.groupby('advertiser').payprice.sum().values
    stats['Avg CPM'] = (((win.Cost / stats.Impressions)).round(2)).astype(str) 
    return stats

In [18]:
# train data statistics
train_stat = basic_stats(train)

In [19]:
train_stat

,Advertiser,Impressions,Clicks,Cost,CTR,eCPC,Avg CPM
0,1458,492353,385,33968.7,0.078%,88.23,68.99
1,2259,133673,43,12428.2,0.032%,289.03,92.97
2,2261,110122,36,9873.8,0.033%,274.27,89.66
3,2821,211366,131,18828.0,0.062%,143.73,89.08
4,2997,49829,217,3129.3,0.435%,14.42,62.8
5,3358,264956,202,22447.2,0.076%,111.12,84.72
6,3386,455041,320,34931.8,0.07%,109.16,76.77
7,3427,402806,272,30458.7,0.068%,111.98,75.62
8,3476,310835,187,23918.8,0.06%,127.91,76.95


In [64]:
print(train_stat.to_latex())

\begin{tabular}{lrrrrlll}
\toprule
{} &  Advertiser &  Impressions &  Clicks &     Cost &     CTR &    eCPC & Avg CPM \\
\midrule
0 &        1458 &       492353 &     385 &  33968.7 &  0.078\% &   88.23 &   68.99 \\
1 &        2259 &       133673 &      43 &  12428.2 &  0.032\% &  289.03 &   92.97 \\
2 &        2261 &       110122 &      36 &   9873.8 &  0.033\% &  274.27 &   89.66 \\
3 &        2821 &       211366 &     131 &  18828.0 &  0.062\% &  143.73 &   89.08 \\
4 &        2997 &        49829 &     217 &   3129.3 &  0.435\% &   14.42 &    62.8 \\
5 &        3358 &       264956 &     202 &  22447.2 &  0.076\% &  111.12 &   84.72 \\
6 &        3386 &       455041 &     320 &  34931.8 &   0.07\% &  109.16 &   76.77 \\
7 &        3427 &       402806 &     272 &  30458.7 &  0.068\% &  111.98 &   75.62 \\
8 &        3476 &       310835 &     187 &  23918.8 &   0.06\% &  127.91 &   76.95 \\
\bottomrule
\end{tabular}



In [20]:
train_stat.to_csv('train_stat.csv', header=True, index_label='advertiserid')

In [21]:
val_stats = basic_stats(validation)

In [22]:
val_stats

,Advertiser,Impressions,Clicks,Cost,CTR,eCPC,Avg CPM
0,1458,62353,49,4294.6,0.079%,87.64,68.88
1,2259,16715,2,1568.8,0.012%,784.4,93.86
2,2261,13550,3,1214.9,0.022%,404.96,89.66
3,2821,26503,23,2394.9,0.087%,104.13,90.36
4,2997,6176,26,388.8,0.421%,14.95,62.95
5,3358,32939,23,2794.0,0.07%,121.48,84.82
6,3386,56665,28,4350.8,0.049%,155.39,76.78
7,3427,50183,37,3776.7,0.074%,102.07,75.26
8,3476,38841,11,2993.8,0.028%,272.16,77.08


In [66]:
print(val_stats.to_latex())

\begin{tabular}{lrrrrlll}
\toprule
{} &  Advertiser &  Impressions &  Clicks &    Cost &     CTR &    eCPC & Avg CPM \\
\midrule
0 &        1458 &        62353 &      49 &  4294.6 &  0.079\% &   87.64 &   68.88 \\
1 &        2259 &        16715 &       2 &  1568.8 &  0.012\% &   784.4 &   93.86 \\
2 &        2261 &        13550 &       3 &  1214.9 &  0.022\% &  404.96 &   89.66 \\
3 &        2821 &        26503 &      23 &  2394.9 &  0.087\% &  104.13 &   90.36 \\
4 &        2997 &         6176 &      26 &   388.8 &  0.421\% &   14.95 &   62.95 \\
5 &        3358 &        32939 &      23 &  2794.0 &   0.07\% &  121.48 &   84.82 \\
6 &        3386 &        56665 &      28 &  4350.8 &  0.049\% &  155.39 &   76.78 \\
7 &        3427 &        50183 &      37 &  3776.7 &  0.074\% &  102.07 &   75.26 \\
8 &        3476 &        38841 &      11 &  2993.8 &  0.028\% &  272.16 &   77.08 \\
\bottomrule
\end{tabular}



In [9]:
val_stats.to_csv('val_stats.csv', header=True, index_label='advertiserid')